# Scraping apple products prices and review info from www.abt.com

In [102]:
import requests
from bs4 import BeautifulSoup
import time
import random
import csv
import pandas as pd
import numpy as np
import re 
import mariadb
import sys

## Exploration and research

This part before the next heading shows the main part of process of researching or finding a solution, is not part of the processing itself and can be skipped. 

In [103]:
# testing with one of urls: 
url = 'https://www.abt.com/Apple-Cell-Phones/bc/281-1228.html'
r = requests.get(url)
doc = BeautifulSoup(r.text, "html.parser")

In [104]:
doc.select_one('.productPageLink').attrs['href']

'https://www.abt.com/Apple-128GB-Deep-Purple-iPhone-14-Pro-Cellular-Phone-MQ0E3LLA6514D/p/183648.html'

Scraping product page:

In [105]:
def scrape_prods(url): 
    r = requests.get(url)
    doc = BeautifulSoup(r.text, "html.parser")
    products = doc.select(".category_item_container")
    i = 1
    for product in products: 
        title = product.select_one('.cl_title').text.strip()
        price = product.select_one('.pricing-item-price').text.strip()
        print(i)
        print(title)
        print(price)
        if product.select_one('.abt-reviews-rating'):
            rating = product.select_one('.abt-reviews-rating .sr-only').text.strip()
            review = product.select_one('.abt-reviews-num-reviews').text.strip()
            print(rating)
            print(review)
        i +=1
    
scrape_prods(url)

1
Apple 128GB Deep Purple iPhone 14 Pro Cellular Phone - MQ0E3LL/A & 6514D
Your Price: $999.99
2
Apple 128GB Space Black iPhone 14 Pro Max Cellular Phone - MQ8N3LL/A & 6527D
Your Price: $1,099.99
3
Apple 128GB Silver iPhone 14 Pro Max Cellular Phone - MQ8P3LL/A & 6528D
Your Price: $1,099.99
4
Apple 64GB (PRODUCT)RED iPhone SE (3rd Generation) Cellular Phone - 6372D
Your Price: $429.99
5
Apple 256GB Starlight iPhone 14 Cellular Phone - MPW23LL/A & 6487D
Your Price: $899.99
6
Apple 128GB Blue iPhone 14 Plus Cellular Phone - MQ3W3LL/A & 6500D
Your Price: $899.99
7
Apple 256GB (PRODUCT)RED iPhone 14 Plus Cellular Phone - MQ413LL/A & 6504D
Your Price: $999.99
5 out of 5 stars
Contains 1 Reviews
8
Apple 256GB Purple iPhone 14 Cellular Phone - MPW83LL/A & 6488D
Your Price: $899.99
9
Apple 256GB Space Black iPhone 14 Pro Cellular Phone - MQ0N3LL/A & 6515D
Your Price: $1,099.99
5 out of 5 stars
Contains 1 Reviews
10
Apple 64GB Starlight iPhone SE (3rd Generation) Cellular Phone - 6371D
Your Pri

In [106]:
#scrpaing URL of next page of category: 

nextbutton = "Next »"  
links = doc.find_all('a', string=nextbutton)
if links:
    print(links[0].attrs['href'])
else: 
    print('It was last page')    

https://www.abt.com/Apple-Cell-Phones/bc/281-1228.html?category_id=1228&start_index=20


In [107]:
#Function for scraping next URL without arguments
def next_url():
    global url
    sl = random.randint(1,15)
    print(sl)
    time.sleep(sl)
    r = requests.get(url)
    doc = BeautifulSoup(r.text, "html.parser")
    nextbutton = "Next »"  
    links = doc.find_all('a', string=nextbutton)
    if links:
        url = links[0].attrs['href']
    else: url = 'done'
    print(url)

next_url()
url

1
https://www.abt.com/Apple-Cell-Phones/bc/281-1228.html?category_id=1228&start_index=20


'https://www.abt.com/Apple-Cell-Phones/bc/281-1228.html?category_id=1228&start_index=20'

In [96]:
# getting next URL with argument avoidin Global variables

def next_url2(url):
    sl = random.randint(1,10)
    print(sl)
    time.sleep(sl)
    r = requests.get(url)
    doc = BeautifulSoup(r.text, "html.parser")
    nextbutton = "Next »"  
    links = doc.find_all('a', string=nextbutton)
    if links:
        url = links[0].attrs['href']
    else: url = 'done'
    print(url)
    return(url)

next_url2(url)

8
https://www.abt.com/Apple-Cell-Phones/bc/281-1228.html?category_id=1228&start_index=20


'https://www.abt.com/Apple-Cell-Phones/bc/281-1228.html?category_id=1228&start_index=20'

In [108]:
# test to scrape all iphones: 

while url != 'done': 
    scrape_prods(url)
    next_url()

1
Apple 256GB Blue iPhone 14 Plus Cellular Phone - MQ423LL/A & 6505D
Your Price: $999.99
5 out of 5 stars
Contains 1 Reviews
2
Apple 256GB Midnight iPhone 14 Plus Cellular Phone - MQ3X3LL/A & 6501D
Your Price: $999.99
5 out of 5 stars
Contains 1 Reviews
3
Apple 64GB Midnight iPhone SE (3rd Generation) Cellular Phone - 6370D
Your Price: $429.99
4
Apple 128GB Midnight iPhone SE (3rd Generation) Cellular Phone - 6373D
Your Price: $479.99
5
Apple 128GB (PRODUCT)RED iPhone 14 Plus Cellular Phone - MQ3V3LL/A & 6499D
Your Price: $899.99
6
Apple 128GB (PRODUCT)RED iPhone SE (3rd Generation) Cellular Phone - 6375D
Your Price: $479.99
7
Apple 256GB (PRODUCT)RED iPhone 14 Cellular Phone - MPWF3LL/A & 6489D
Your Price: $899.99
8
Apple 128GB Deep Purple iPhone 14 Pro Max Cellular Phone - MQ8R3LL/A & 6530D
Your Price: $1,099.99
9
Apple 256GB Blue iPhone 14 Cellular Phone - MPWM3LL/A & 6490D
Your Price: $899.99
10
Apple 256GB Purple iPhone 14 Plus Cellular Phone - MQ403LL/A & 6503D
Your Price: $999.9

done


It is working solution but in this case we need to call the same URL twice wich is not the best idea. So below I combined two functions in one. 

## Automatization of getting URLs to scrape
This part describe the aotomated process of gathering URLs to scrape


First step is to scrape URLs of categories form start page:

In [109]:
# Starting page with links to category pages and pages with links to other categories: 
start_url = 'https://www.abt.com/brand/apple'

r = requests.get(start_url)
doc = BeautifulSoup(r.text, "html.parser")
url_list = doc.select('.cat_list_drop')

pages_to_check = {} #dictionary (in order to avoid duplicates) with list of names of category and corresponding url.
for url in url_list:
    pages_to_check[url.text] = url.attrs['href']

pages_to_check

{'Computers & Tablets': 'https://www.abt.com/Apple-Computers-Tablets/bc/281-414.html',
 'Headphones': 'https://www.abt.com/Apple-Headphones/bc/281-2089.html',
 'Phones': 'https://www.abt.com/Apple-Phones/bc/281-9.html',
 'Speakers': 'https://www.abt.com/Apple-Speakers/bc/281-26.html',
 'TVs & Home Theater': 'https://www.abt.com/Apple-TVs-Home-Theater/bc/281-2261.html',
 'Watches': 'https://www.abt.com/Apple-Watches/bc/281-995.html',
 'Wearables': 'https://www.abt.com/Apple-Wearables/bc/281-2479.html',
 'Audio & Video Accessories': 'https://www.abt.com/Apple-Audio-Video-Accessories/bc/281-59.html',
 'Cell Phone Accessories': 'https://www.abt.com/Apple-Cell-Phone-Accessories/bc/281-301.html',
 'Computer Accessories': 'https://www.abt.com/Apple-Computer-Accessories/bc/281-481.html',
 'iPad & Tablet Accessories': 'https://www.abt.com/Apple-iPad-Tablet-Accessories/bc/281-2063.html',
 'Watch Accessories': 'https://www.abt.com/Apple-Watch-Accessories/bc/281-1017.html'}

In [110]:
#final 'list' (not in pythonic meaning) with URLs of categories to scrape will be the dictionary:
pages_to_scrape = {}

for page in pages_to_check: #going throigh initial list
    url = pages_to_check[page] 
    sl = random.randint(1,5) # in order to be even more unpredictable and not to be blocked by cloudflare
    print('waiting for ', sl, 'sec')
    time.sleep(sl)
    doc = BeautifulSoup(requests.get(url).text, "html.parser")
    if doc.select_one('.category_item_container'): #if page contains block of products (items) than copy it 
        #to final list of pages to scrape 
        pages_to_scrape[url] = page # in this dictinory URLs are keys, not Category as in previous, 
        # because I want store only unique URLs, and categories could have different names probably because of SEO goals
    else: # adding pages with subcategories to list (dictionary) with pages to scrape
        subcat_links = doc.select('.category-block-link')
        for link in subcat_links:
            pages_to_scrape[link.attrs['href']] = link.text.strip()
    print('Categories to scrape: ', len(pages_to_scrape))

for url in pages_to_scrape: 
    print('Category: ', pages_to_scrape[url], 'url: ', url)


waiting for  1 sec
Categories to scrape:  7
waiting for  5 sec
Categories to scrape:  12
waiting for  2 sec
Categories to scrape:  14
waiting for  3 sec
Categories to scrape:  18
waiting for  5 sec
Categories to scrape:  21
waiting for  5 sec
Categories to scrape:  24
waiting for  4 sec
Categories to scrape:  27
waiting for  4 sec
Categories to scrape:  30
waiting for  2 sec
Categories to scrape:  36
waiting for  1 sec
Categories to scrape:  40
waiting for  2 sec
Categories to scrape:  44
waiting for  4 sec
Categories to scrape:  45
Category:  Apple Desktop Computers url:  https://www.abt.com/Apple-Desktop-Computers/bc/281-444.html
Category:  Apple Laptops & Notebook Computers url:  https://www.abt.com/Apple-Laptops-Notebook-Computers/bc/281-415.html
Category:  Apple iPads url:  https://www.abt.com/Apple-iPads/bc/281-2367.html
Category:  Apple Tablets url:  https://www.abt.com/Apple-Tablets/bc/281-2016.html
Category:  Apple Computer Monitors url:  https://www.abt.com/Apple-Computer-Mon

After this procedure there still could be pages with subcategories not products in dictionary (with SEO purposes), 
but it is ok for us since we have all necessary pages with products including main categories: phones, tablets, 
and laptops. Also important to add that I avoided to scrape same pages twice because of using dictionary. 

## Scraping into CSV 

In [112]:
# function to scrape products and next url altogether, and write the result at once in CSV

def scrape_prods_to_csv(url, category): #category will be stored in CSV as well, because it will be used on next steps
    print(category)
    with open('../data/abt_rudyka_stage1.csv', 'a', newline='') as csvfile: #method is in order to just mini
        writer = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL) #quoting= just in case we have ';' somewhere
        sleep = random.randint(1,5) 
        print('Waiting for', sleep, 'and then scrape', url) # to be informed of the process
        time.sleep(sleep)
        r = requests.get(url)
        doc = BeautifulSoup(r.text, "html.parser")
        products = doc.select(".category_item_container")
        for product in products: 
            title = product.select_one('.cl_title').text.strip() #title of the product
            price = product.select_one('.pricing-item-price').text.strip() #all the content of tag with price 
            product_url = product.select_one('.productPageLink').attrs['href'] #url of product page
            try:
                rating = product.select_one('.abt-reviews-rating .sr-only').text.strip()
                review = product.select_one('.abt-reviews-num-reviews').text.strip()
            except AttributeError:
                rating = review = None
            writer.writerow([url, category, title, price, product_url, rating, review]) 
    nextbutton = "Next »"  #scraping URL of next page
    links = doc.find_all('a', string=nextbutton) #list of urls with only one element
    if links:
        url = links[0].attrs['href']
        print('naxt page to scrape: ', url)
        scrape_prods_to_csv(url, category) #simple recursion. Because why not     

## Scrpaing all the needed pages for products properties one by one: 

In [113]:
for url, cat in pages_to_scrape.items():  
    scrape_prods_to_csv(url, cat)

print('The end')

Apple Desktop Computers
Waiting for 5 and then scrape https://www.abt.com/Apple-Desktop-Computers/bc/281-444.html
naxt page to scrape:  https://www.abt.com/Apple-Desktop-Computers/bc/281-444.html?category_id=444&start_index=20
Apple Desktop Computers
Waiting for 2 and then scrape https://www.abt.com/Apple-Desktop-Computers/bc/281-444.html?category_id=444&start_index=20
Apple Laptops & Notebook Computers
Waiting for 5 and then scrape https://www.abt.com/Apple-Laptops-Notebook-Computers/bc/281-415.html
naxt page to scrape:  https://www.abt.com/Apple-Laptops-Notebook-Computers/bc/281-415.html?category_id=415&start_index=20
Apple Laptops & Notebook Computers
Waiting for 4 and then scrape https://www.abt.com/Apple-Laptops-Notebook-Computers/bc/281-415.html?category_id=415&start_index=20
Apple iPads
Waiting for 3 and then scrape https://www.abt.com/Apple-iPads/bc/281-2367.html
naxt page to scrape:  https://www.abt.com/Apple-iPads/bc/281-2367.html?category_id=2367&start_index=20
Apple iPads
W

Apple Wearable Technology
Waiting for 5 and then scrape https://www.abt.com/Apple-Wearable-Technology/bc/281-2351.html
naxt page to scrape:  https://www.abt.com/Apple-Wearable-Technology/bc/281-2351.html?category_id=2351&start_index=20
Apple Wearable Technology
Waiting for 3 and then scrape https://www.abt.com/Apple-Wearable-Technology/bc/281-2351.html?category_id=2351&start_index=20
naxt page to scrape:  https://www.abt.com/Apple-Wearable-Technology/bc/281-2351.html?category_id=2351&start_index=40
Apple Wearable Technology
Waiting for 5 and then scrape https://www.abt.com/Apple-Wearable-Technology/bc/281-2351.html?category_id=2351&start_index=40
naxt page to scrape:  https://www.abt.com/Apple-Wearable-Technology/bc/281-2351.html?category_id=2351&start_index=60
Apple Wearable Technology
Waiting for 1 and then scrape https://www.abt.com/Apple-Wearable-Technology/bc/281-2351.html?category_id=2351&start_index=60
naxt page to scrape:  https://www.abt.com/Apple-Wearable-Technology/bc/281-23

# Stage 2: impurifying

## Checking the result

In [115]:
!head -n 5 ../data/abt_rudyka_stage1.csv #checking the stored file if it is ok 

In [116]:
#reading the file in pandas DataFrame object
abt = pd.read_csv('../data/abt_rudyka_stage1.csv', sep =';', names = ['scraped_url', 'category', 'title', 'price', 'product_url', 'rating', 'review'])

In [117]:
abt.head(10)

,scraped_url,category,title,price,product_url,rating,review
0,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Chip 8GB Unifi...,Sale $557.00,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,5 out of 5 stars,Contains 1 Reviews
1,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Silver 24"" iMac M1 8-Core 8GB RAM 256GB ...","Your Price: $1,299.00",https://www.abt.com/Apple-Silver-24-iMac-M1-8-...,5 out of 5 stars,Contains 4 Reviews
2,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Blue 24"" iMac M1 8-Core 8GB RAM 256GB SS...","Your Price: $1,299.00",https://www.abt.com/Apple-Blue-24-iMac-M1-8-Co...,5 out of 5 stars,Contains 4 Reviews
3,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Studio Desktop Apple M1 Ultra Chip 6...,"Your Price: $3,999.00",https://www.abt.com/Apple-Mac-Studio-Desktop-A...,NaN,NaN
4,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Green 24"" iMac M1 8-Core 8GB RAM 256GB S...","Your Price: $1,299.00",https://www.abt.com/Apple-Green-24-iMac-M1-8-C...,5 out of 5 stars,Contains 4 Reviews
5,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Pink 24"" iMac M1 8-Core 8GB RAM 256GB SS...","Your Price: $1,499.00",https://www.abt.com/Apple-Pink-24-iMac-M1-8-Co...,5 out of 5 stars,Contains 1 Reviews
6,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Pro Chip 16GB ...,"Your Price: $1,299.00",https://www.abt.com/Apple-Mac-Mini-Desktop-App...,NaN,NaN
7,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Chip 8GB Unifi...,Your Price: $799.00,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,NaN,NaN
8,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M1 Chip 8GB Unifi...,Sale $869.00,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,5 out of 5 stars,Contains 3 Reviews
9,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Pink 24"" iMac M1 8-Core 8GB RAM 512GB SS...","Your Price: $1,699.00",https://www.abt.com/Apple-Pink-24-iMac-M1-8-Co...,5 out of 5 stars,Contains 1 Reviews


In [19]:
#check of duplicates using should be unique url of products 
sum(abt['product_url'].duplicated())

405

## Additional impurification

Data already has natural impurities:  duplicates, NA values, incorrect types, irrelevant data or contaminated data for some values,  strings with words in prices (e.g. 'Your Price: $799.00'), reviews and ratings ('5 out of 5 stars') where only numbers needed. But this is only 4 differnt types of impurites. Need at least one more. So I impurified by increasing some prices by removing points in float number and deleting Titles of some products in order to recreate it later using URL of tha corresponding product page. 

### Changing prices to get outliers/misspels

In [119]:
# impurifying prices by removing dots, after that we can consider following prices as outliers and/or misspellings 
len(abt)

# 800+ rows so we need 85 errors to get 10 errors for 100
for i in range(1, 86):
    row = random.randint(1,805)
    abt.loc[row,'price'] = abt.loc[row,'price'].replace('.','')


In [120]:
abt.head(30)

,scraped_url,category,title,price,product_url,rating,review
0,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Chip 8GB Unifi...,Sale $557.00,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,5 out of 5 stars,Contains 1 Reviews
1,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Silver 24"" iMac M1 8-Core 8GB RAM 256GB ...","Your Price: $1,299.00",https://www.abt.com/Apple-Silver-24-iMac-M1-8-...,5 out of 5 stars,Contains 4 Reviews
2,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Blue 24"" iMac M1 8-Core 8GB RAM 256GB SS...","Your Price: $1,299.00",https://www.abt.com/Apple-Blue-24-iMac-M1-8-Co...,5 out of 5 stars,Contains 4 Reviews
3,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Studio Desktop Apple M1 Ultra Chip 6...,"Your Price: $3,99900",https://www.abt.com/Apple-Mac-Studio-Desktop-A...,NaN,NaN
4,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Green 24"" iMac M1 8-Core 8GB RAM 256GB S...","Your Price: $1,299.00",https://www.abt.com/Apple-Green-24-iMac-M1-8-C...,5 out of 5 stars,Contains 4 Reviews
5,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Pink 24"" iMac M1 8-Core 8GB RAM 256GB SS...","Your Price: $1,499.00",https://www.abt.com/Apple-Pink-24-iMac-M1-8-Co...,5 out of 5 stars,Contains 1 Reviews
6,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Pro Chip 16GB ...,"Your Price: $1,299.00",https://www.abt.com/Apple-Mac-Mini-Desktop-App...,NaN,NaN
7,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Chip 8GB Unifi...,Your Price: $799.00,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,NaN,NaN
8,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M1 Chip 8GB Unifi...,Sale $869.00,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,5 out of 5 stars,Contains 3 Reviews
9,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Pink 24"" iMac M1 8-Core 8GB RAM 512GB SS...","Your Price: $1,699.00",https://www.abt.com/Apple-Pink-24-iMac-M1-8-Co...,5 out of 5 stars,Contains 1 Reviews


### Removing titles to add additional NAs

In [123]:
#removing some of titles (in order to recreate them then from url later, 30 would be enough

for i in range(1, 30):
    row = random.randint(1,805)
    abt.loc[row,'title'] = np.nan


In [124]:
len(abt[abt.title.isnull()])

84

In [26]:
abt.to_csv('../data/abt_rudyka_stage2.csv')

# Stage 3: Cleaning, preparation for merging, adding calculations

In [125]:
abt = pd.read_csv('../data/abt_rudyka_stage2.csv', sep =',', index_col = 0)

Clearing prices

In [126]:
abt.head()

,scraped_url,category,title,price,product_url,rating,review
0,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Chip 8GB Unifi...,Sale $557.00,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,5 out of 5 stars,Contains 1 Reviews
1,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Silver 24"" iMac M1 8-Core 8GB RAM 256GB ...","Your Price: $1,299.00",https://www.abt.com/Apple-Silver-24-iMac-M1-8-...,5 out of 5 stars,Contains 4 Reviews
2,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Blue 24"" iMac M1 8-Core 8GB RAM 256GB SS...","Your Price: $1,299.00",https://www.abt.com/Apple-Blue-24-iMac-M1-8-Co...,5 out of 5 stars,Contains 4 Reviews
3,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Studio Desktop Apple M1 Ultra Chip 6...,"Your Price: $3,99900",https://www.abt.com/Apple-Mac-Studio-Desktop-A...,NaN,NaN
4,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Green 24"" iMac M1 8-Core 8GB RAM 256GB S...","Your Price: $1,299.00",https://www.abt.com/Apple-Green-24-iMac-M1-8-C...,5 out of 5 stars,Contains 4 Reviews


In [127]:
outliers = ~abt.price.str.contains('\.') #filter with "dirty" prices
sum(outliers)

152

In [128]:
abt.loc[outliers, 'price'] = abt.loc[outliers, 'price'].str[:-2] +'.'+ abt.loc[outliers, 'price'].str[-2:]
# replace(r'(\d)(\d{2})$', r'\1.\2') #we have prices .00 and .99, 
# that is why simple regex was used

In [129]:
abt.head()

,scraped_url,category,title,price,product_url,rating,review
0,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Chip 8GB Unifi...,Sale $557.00,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,5 out of 5 stars,Contains 1 Reviews
1,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Silver 24"" iMac M1 8-Core 8GB RAM 256GB ...","Your Price: $1,299.00",https://www.abt.com/Apple-Silver-24-iMac-M1-8-...,5 out of 5 stars,Contains 4 Reviews
2,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Blue 24"" iMac M1 8-Core 8GB RAM 256GB SS...","Your Price: $1,299.00",https://www.abt.com/Apple-Blue-24-iMac-M1-8-Co...,5 out of 5 stars,Contains 4 Reviews
3,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Studio Desktop Apple M1 Ultra Chip 6...,"Your Price: $3,999.00",https://www.abt.com/Apple-Mac-Studio-Desktop-A...,NaN,NaN
4,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Green 24"" iMac M1 8-Core 8GB RAM 256GB S...","Your Price: $1,299.00",https://www.abt.com/Apple-Green-24-iMac-M1-8-C...,5 out of 5 stars,Contains 4 Reviews


In [130]:
abt[~abt.price.str.contains('\.')] #despite warning everything is fine

,scraped_url,category,title,price,product_url,rating,review


Recreating title from url 

In [131]:
no_title = abt.title.isnull()
name_pattern = r'.com\/(.+)\/p.+' #everything after '.com/' and before next '/'

new_titles = abt[no_title].product_url.str.extract(name_pattern).replace('-',' ', regex = True) 
# variable was createdto avoid /var/folders/3t/1cf4qhj170q46d820lvp7bzc0000gn/T/ipykernel_21966/891614863.py:6: SettingWithCopyWarning: 

abt.loc[no_title, ['title']] = new_titles[0] # .loc was used to avoid ValueError: Incompatible indexer with DataFrame

In [132]:
sum(abt.title.isnull())

0

Removing Duplicates

In [168]:
abt.drop_duplicates('product_url', inplace = True) #url of product should be unique
len(abt)

484

## Further cleaning and extracting values

We're going to use unique model identifier for merging data for everything, except iphones, which don't have such ids on the second Swiss website, so for phone we're going to extract model name and storage. 

### Exploration of function and patterns

This part before the next heading shows the part of process of researching or finding a proper way for exctaction needed data out of text, is not part of the processing itself and can be skipped. 

In [133]:
model_name_pattern = r'(([A-Z]|\d|\/){9})' #model id always has 9 digits or uppercase letters

In [134]:
model_name_reg = re.compile(model_name_pattern) #compliling to improve performance

In [135]:
# checking with simple text

text = '''
Apple Watch Series 8 GPS & Cellular 45mm Gold Stainless Steel Case With S/M Starlight Sport Band - MNVX3LL/A
Your Price: $749.00
7
Apple Watch Series 8 GPS & Cellular 41mm Silver Stainless Steel Case With M/L White Sport Band - MNV83LL/A
Your Price: $699.00
'''

model_name_reg.search(text).group()

'MNVX3LL/A'

In [136]:
#extracting model name wih applying lambda function: 
abt['model_id'] = abt['title'].apply(lambda x: model_name_reg.search(x).group() if model_name_reg.search(x) else None)


In [39]:
abt.head(40) #checking if that worked

,scraped_url,category,title,price,product_url,rating,review,model_id
0,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Chip 8GB Unifi...,Sale $557.00,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,5 out of 5 stars,Contains 1 Reviews,MMFJ3LL/A
1,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Silver 24"" iMac M1 8-Core 8GB RAM 256GB ...","Your Price: $1,299.00",https://www.abt.com/Apple-Silver-24-iMac-M1-8-...,5 out of 5 stars,Contains 4 Reviews,MGTF3LL/A
2,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Blue 24"" iMac M1 8-Core 8GB RAM 256GB SS...","Your Price: $1,299.00",https://www.abt.com/Apple-Blue-24-iMac-M1-8-Co...,5 out of 5 stars,Contains 4 Reviews,MJV93LL/A
3,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Studio Desktop Apple M1 Ultra Chip 6...,"Your Price: $3,999.00",https://www.abt.com/Apple-Mac-Studio-Desktop-A...,NaN,NaN,MJMW3LL/A
4,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Green 24"" iMac M1 8-Core 8GB RAM 256GB S...","Your Price: $1,299.00",https://www.abt.com/Apple-Green-24-iMac-M1-8-C...,5 out of 5 stars,Contains 4 Reviews,MJV83LL/A
5,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Pink 24"" iMac M1 8-Core 8GB RAM 256GB SS...","Your Price: $1,499.00",https://www.abt.com/Apple-Pink-24-iMac-M1-8-Co...,5 out of 5 stars,Contains 1 Reviews,MGPM3LL/A
6,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Pro Chip 16GB ...,"Your Price: $1,299.00",https://www.abt.com/Apple-Mac-Mini-Desktop-App...,NaN,NaN,MNH73LL/A
7,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Chip 8GB Unifi...,Sale $743.00,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,NaN,NaN,MMFK3LL/A
8,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M1 Chip 8GB Unifi...,Sale $869.00,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,5 out of 5 stars,Contains 3 Reviews,MGNT3LL/A
9,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Pink 24"" iMac M1 8-Core 8GB RAM 512GB SS...","Your Price: $1,699.00",https://www.abt.com/Apple-Pink-24-iMac-M1-8-Co...,5 out of 5 stars,Contains 1 Reviews,MGPN3LL/A


### Extracting Model identifier (real process)

In [137]:
abt['model_id'] = abt['title'].str.extract(r'([A-Z\d\/]{9})') #used str.extract() method as more readable 

### Extracting model and storage for iPhones

In [139]:
# since scraped data from another website does not have unique model numbers for iPhones, 
# We decided to exclude separately for iphone: model (13, 13 pro max), and storage in separate columns

pd.set_option('display.max_colwidth', None) # to show titles fully 
iphones = abt[abt['title'].str.contains('iPhone')]

iphones_filtered = abt['title'].str.contains('iPhone')
iphones['title'].head(10)

300        Apple 128GB Deep Purple iPhone 14 Pro Cellular Phone - MQ0E3LL/A & 6514D
301    Apple 128GB Space Black iPhone 14 Pro Max Cellular Phone - MQ8N3LL/A & 6527D
302         Apple 128GB Silver iPhone 14 Pro Max Cellular Phone - MQ8P3LL/A & 6528D
303       Apple 64GB (PRODUCT)RED iPhone SE (3rd Generation) Cellular Phone - 6372D
304              Apple 256GB Starlight iPhone 14 Cellular Phone - MPW23LL/A & 6487D
305              Apple 128GB Blue iPhone 14 Plus Cellular Phone - MQ3W3LL/A & 6500D
306      Apple 256GB (PRODUCT)RED iPhone 14 Plus Cellular Phone - MQ413LL/A & 6504D
307                 Apple 256GB Purple iPhone 14 Cellular Phone - MPW83LL/A & 6488D
308        Apple 256GB Space Black iPhone 14 Pro Cellular Phone - MQ0N3LL/A & 6515D
309          Apple 64GB Starlight iPhone SE (3rd Generation) Cellular Phone - 6371D
Name: title, dtype: object

In [140]:
abt[~iphones_filtered].head(5)

,scraped_url,category,title,price,product_url,rating,review,model_id
0,https://www.abt.com/Apple-Desktop-Computers/bc/281-444.html,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Chip 8GB Unified RAM 256GB SSD (Early 2023) - MMFJ3LL/A,Sale $557.00,https://www.abt.com/Apple-Mac-Mini-Desktop-Apple-M2-Chip-8GB-Unified-RAM-256GB-SSD-Early-2023-MMFJ3LLA/p/188787.html,5 out of 5 stars,Contains 1 Reviews,MMFJ3LL/A
1,https://www.abt.com/Apple-Desktop-Computers/bc/281-444.html,Apple Desktop Computers,"Apple Silver 24"" iMac M1 8-Core 8GB RAM 256GB SSD, 7-Core GPU Retina 4.5K Desktop Computer (2021) - MGTF3LL/A","Your Price: $1,299.00",https://www.abt.com/Apple-Silver-24-iMac-M1-8-Core-8GB-RAM-256GB-SSD-7-Core-GPU-Retina-4.5K-Desktop-Computer-2021-MGTF3LLA/p/163250.html,5 out of 5 stars,Contains 4 Reviews,MGTF3LL/A
2,https://www.abt.com/Apple-Desktop-Computers/bc/281-444.html,Apple Desktop Computers,"Apple Blue 24"" iMac M1 8-Core 8GB RAM 256GB SSD, 7-Core GPU Retina 4.5K Desktop Computer (2021) - MJV93LL/A","Your Price: $1,299.00",https://www.abt.com/Apple-Blue-24-iMac-M1-8-Core-8GB-RAM-256GB-SSD-7-Core-GPU-Retina-4.5K-Desktop-Computer-2021-MJV93LLA/p/163252.html,5 out of 5 stars,Contains 4 Reviews,MJV93LL/A
3,https://www.abt.com/Apple-Desktop-Computers/bc/281-444.html,Apple Desktop Computers,Apple Mac Studio Desktop Apple M1 Ultra Chip 64GB Unified RAM 1TB SSD (Early 2022) - MJMW3LL/A,"Your Price: $3,999.00",https://www.abt.com/Apple-Mac-Studio-Desktop-Apple-M1-Ultra-Chip-64GB-Unified-RAM-1TB-SSD-Early-2022-MJMW3LLA/p/176177.html,NaN,NaN,MJMW3LL/A
4,https://www.abt.com/Apple-Desktop-Computers/bc/281-444.html,Apple Desktop Computers,"Apple Green 24"" iMac M1 8-Core 8GB RAM 256GB SSD, 7-Core GPU Retina 4.5K Desktop Computer (2021) - MJV83LL/A","Your Price: $1,299.00",https://www.abt.com/Apple-Green-24-iMac-M1-8-Core-8GB-RAM-256GB-SSD-7-Core-GPU-Retina-4.5K-Desktop-Computer-2021-MJV83LLA/p/163251.html,5 out of 5 stars,Contains 4 Reviews,MJV83LL/A


In [141]:
pd.reset_option('display.max_colwidth')

#categories with word iphone
iphones['category'].unique()

array(['Apple Cell Phones', 'Apple iPhone Accessories',
       'Apple Cell Phone Cases', 'Apple Portable Chargers/Power Banks',
       'Apple Wireless Charging'], dtype=object)

We can see not only iphones but also accesories, so then we need to mark somehow only iphones. Then let's choose only Cell Phones category

In [142]:
cellphones_filter = abt['category'] == 'Apple Cell Phones'
iphones = abt[cellphones_filter]
iphones

,scraped_url,category,title,price,product_url,rating,review,model_id
300,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 128GB Deep Purple iPhone 14 Pro Cellular...,Your Price: $999.99,https://www.abt.com/Apple-128GB-Deep-Purple-iP...,NaN,NaN,MQ0E3LL/A
301,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 128GB Space Black iPhone 14 Pro Max Cell...,"Your Price: $1,099.99",https://www.abt.com/Apple-128GB-Space-Black-iP...,NaN,NaN,MQ8N3LL/A
302,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 128GB Silver iPhone 14 Pro Max Cellular ...,"Your Price: $1,099.99",https://www.abt.com/Apple-128GB-Silver-iPhone-...,NaN,NaN,MQ8P3LL/A
303,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 64GB (PRODUCT)RED iPhone SE (3rd Generat...,Your Price: $429.99,https://www.abt.com/Apple-64GB-PRODUCT-RED-iPh...,NaN,NaN,NaN
304,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 256GB Starlight iPhone 14 Cellular Phone...,Your Price: $899.99,https://www.abt.com/Apple-256GB-Starlight-iPho...,NaN,NaN,MPW23LL/A
...,...,...,...,...,...,...,...,...
395,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 512GB Midnight iPhone 13 Cellular Phone ...,"Your Price: $1,029.99",https://www.abt.com/Apple-512GB-Midnight-iPhon...,NaN,NaN,ML9Q3LL/A
396,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 128GB Starlight iPhone 13 Cellular Phone...,Your Price: $729.99,https://www.abt.com/Apple-128GB-Starlight-iPho...,5 out of 5 stars,Contains 1 Reviews,ML953LLA6
397,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 64GB Blue iPhone 12 Cellular Phone - MGE...,Your Price: $629.99,https://www.abt.com/Apple-64GB-Blue-iPhone-12-...,5 out of 5 stars,Contains 5 Reviews,MGEK3LL/A
398,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 128GB Blue iPhone 12 Cellular Phone - MG...,Your Price: $679.99,https://www.abt.com/Apple-128GB-Blue-iPhone-12...,4 out of 5 stars,Contains 1 Reviews,MGEV3LL/A


In [143]:
# Extracting iphones storage' 
abt['iphone_storage'] = abt.loc[cellphones_filter,'title'].str.extract(r'(\d+[GT]B)')

In [144]:
# Extracting model name
abt['iphone_model'] = abt.loc[cellphones_filter,'title'].str.extract(r'iPhone (.+) Cellular')

In [145]:
# Extracting color
abt['iphone_color'] = abt.loc[cellphones_filter,'title'].str.extract(r'[GT]B (.+) iPhone')

In [146]:
abt[cellphones_filter].head(5)

,scraped_url,category,title,price,product_url,rating,review,model_id,iphone_storage,iphone_model,iphone_color
300,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 128GB Deep Purple iPhone 14 Pro Cellular...,Your Price: $999.99,https://www.abt.com/Apple-128GB-Deep-Purple-iP...,NaN,NaN,MQ0E3LL/A,128GB,14 Pro,Deep Purple
301,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 128GB Space Black iPhone 14 Pro Max Cell...,"Your Price: $1,099.99",https://www.abt.com/Apple-128GB-Space-Black-iP...,NaN,NaN,MQ8N3LL/A,128GB,14 Pro Max,Space Black
302,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 128GB Silver iPhone 14 Pro Max Cellular ...,"Your Price: $1,099.99",https://www.abt.com/Apple-128GB-Silver-iPhone-...,NaN,NaN,MQ8P3LL/A,128GB,14 Pro Max,Silver
303,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 64GB (PRODUCT)RED iPhone SE (3rd Generat...,Your Price: $429.99,https://www.abt.com/Apple-64GB-PRODUCT-RED-iPh...,NaN,NaN,NaN,64GB,SE (3rd Generation),(PRODUCT)RED
304,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 256GB Starlight iPhone 14 Cellular Phone...,Your Price: $899.99,https://www.abt.com/Apple-256GB-Starlight-iPho...,NaN,NaN,MPW23LL/A,256GB,14,Starlight


## Exctracting and changing types for prices, rating and reviews number

In [147]:
abt['price_cleaned'] = abt['price'].str.extract(r'\$(\d*\,*\d*\.\d+.+)') #extracting digits with commas and dots in price column

In [148]:
#removing ',' and changung type to float
abt['price_cleaned'] = abt['price_cleaned'].str.replace(',','').astype(float)

In [149]:
#extracting number of reviews and average rating
abt['reviews_number'] = abt['review'].str.extract(r'(\d+)').fillna(0).astype(int) #na filled in order to avoid an error

In [150]:
abt['rating_float'] = abt['rating'].str.extract(r'(.{1,3})\sout').astype(float) #1-3 symbols before ' out...'

In [151]:
abt.head() #checking the result

,scraped_url,category,title,price,product_url,rating,review,model_id,iphone_storage,iphone_model,iphone_color,price_cleaned,reviews_number,rating_float
0,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Chip 8GB Unifi...,Sale $557.00,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,5 out of 5 stars,Contains 1 Reviews,MMFJ3LL/A,NaN,NaN,NaN,557.0,1,5.0
1,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Silver 24"" iMac M1 8-Core 8GB RAM 256GB ...","Your Price: $1,299.00",https://www.abt.com/Apple-Silver-24-iMac-M1-8-...,5 out of 5 stars,Contains 4 Reviews,MGTF3LL/A,NaN,NaN,NaN,1299.0,4,5.0
2,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Blue 24"" iMac M1 8-Core 8GB RAM 256GB SS...","Your Price: $1,299.00",https://www.abt.com/Apple-Blue-24-iMac-M1-8-Co...,5 out of 5 stars,Contains 4 Reviews,MJV93LL/A,NaN,NaN,NaN,1299.0,4,5.0
3,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,Apple Mac Studio Desktop Apple M1 Ultra Chip 6...,"Your Price: $3,999.00",https://www.abt.com/Apple-Mac-Studio-Desktop-A...,NaN,NaN,MJMW3LL/A,NaN,NaN,NaN,3999.0,0,NaN
4,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Green 24"" iMac M1 8-Core 8GB RAM 256GB S...","Your Price: $1,299.00",https://www.abt.com/Apple-Green-24-iMac-M1-8-C...,5 out of 5 stars,Contains 4 Reviews,MJV83LL/A,NaN,NaN,NaN,1299.0,4,5.0


## Calculations

To show some calculations, I decided to calculate the price of GB of each device. We actually don't need it in further research but that could be interesting itself. 

Also I added calculation for average price of iPhone model no matter which color and added price in CHF. 

In [153]:
#Addind Cost per GB
abt['gbs'] = abt['title'].str.extract('(\d+)[GT]B(?!.*[GT]B).*').astype(float) #extracting amount of Gb or TB
abt.loc[abt['title'].str.contains('TB'), 'gbs'] *= 1000 # "converting" TB into GB
abt['price_per_gb'] = round(abt.price_cleaned/abt['gbs'],2)

In [156]:
abt.sort_values('price_per_gb').head() #ipads with 2TB storage have the minimum per GB price, not a big surprise

,scraped_url,category,title,price,product_url,rating,review,model_id,iphone_storage,iphone_model,iphone_color,price_cleaned,reviews_number,rating_float,price_per_gb
143,https://www.abt.com/Apple-iPads/bc/281-2367.ht...,Apple iPads,Apple iPad Pro M2 11-Inch 2TB Wi-Fi Silver (20...,"Your Price: $1,899.00",https://www.abt.com/Apple-iPad-Pro-M2-11-Inch-...,NaN,NaN,MNXN3LL/A,NaN,NaN,NaN,1899.0,0,NaN,0.95
145,https://www.abt.com/Apple-iPads/bc/281-2367.ht...,Apple iPads,Apple iPad Pro M2 11-Inch 2TB Wi-Fi Space Gray...,"Your Price: $1,899.00",https://www.abt.com/Apple-iPad-Pro-M2-11-Inch-...,NaN,NaN,MNXM3LL/A,NaN,NaN,NaN,1899.0,0,NaN,0.95
242,https://www.abt.com/Apple-Tablets/bc/281-2016....,Apple Tablets,Apple iPad Pro M2 11-Inch 2TB Wi-Fi Silver (20...,"Your Price: $1,899.00",https://www.abt.com/Apple-iPad-Pro-M2-11-Inch-...,NaN,NaN,MNXN3LL/A,NaN,NaN,NaN,1899.0,0,NaN,0.95
244,https://www.abt.com/Apple-Tablets/bc/281-2016....,Apple Tablets,Apple iPad Pro M2 11-Inch 2TB Wi-Fi Space Gray...,"Your Price: $1,899.00",https://www.abt.com/Apple-iPad-Pro-M2-11-Inch-...,NaN,NaN,MNXM3LL/A,NaN,NaN,NaN,1899.0,0,NaN,0.95
151,https://www.abt.com/Apple-iPads/bc/281-2367.ht...,Apple iPads,Apple iPad Pro M2 11 Inch 2TB Wi Fi Cellular S...,"Your Price: $2,099.00",https://www.abt.com/Apple-iPad-Pro-M2-11-Inch-...,NaN,NaN,NaN,NaN,NaN,NaN,2099.0,0,NaN,1.05


In [62]:
abt.sort_values('price_per_gb', ascending = False).head() 
#the most expensive 1 GB we have in Mac pro and some iPads with minimum storage

,scraped_url,category,title,price,product_url,rating,review,model_id,iphone_storage,iphone_model,iphone_color,price_cleaned,reviews_number,rating_float,gbs,price_per_gb
23,https://www.abt.com/Apple-Desktop-Computers/bc...,Apple Desktop Computers,"Apple Mac Pro 3.5GHz Intel Xeon W 256GB SSD, R...","Your Price: $5,999.00",https://www.abt.com/Apple-Mac-Pro-3.5GHz-Intel...,NaN,NaN,Z0W30003V,NaN,NaN,NaN,5999.00,0,NaN,256.0,23.43
114,https://www.abt.com/Apple-iPads/bc/281-2367.ht...,Apple iPads,Apple iPad Air 64GB Wi-Fi + Cellular Space Gra...,Sale $689.00,https://www.abt.com/Apple-iPad-Air-64GB-Wi-Fi-...,NaN,NaN,MM6R3LL/A,NaN,NaN,NaN,689.00,0,NaN,64.0,10.77
130,https://www.abt.com/Apple-iPads/bc/281-2367.ht...,Apple iPads,Apple iPad mini 8.3-Inch 64GB Wi-Fi + Cellular...,Your Price: $649.99,https://www.abt.com/Apple-iPad-mini-8.3-Inch-6...,4 out of 5 stars,Contains 1 Reviews,MK893LL/A,NaN,NaN,NaN,649.99,1,4.0,64.0,10.16
110,https://www.abt.com/Apple-iPads/bc/281-2367.ht...,Apple iPads,Apple iPad Air 64GB Wi-Fi + Cellular Purple (2...,Sale $649.99,https://www.abt.com/Apple-iPad-Air-64GB-Wi-Fi-...,NaN,NaN,MME93LL/A,NaN,NaN,NaN,649.99,0,NaN,64.0,10.16
113,https://www.abt.com/Apple-iPads/bc/281-2367.ht...,Apple iPads,Apple iPad Air 64GB Wi-Fi + Cellular Starlight...,Sale $649.99,https://www.abt.com/Apple-iPad-Air-64GB-Wi-Fi-...,NaN,NaN,MM6V3LL/A,NaN,NaN,NaN,649.99,0,NaN,64.0,10.16


In [157]:
abt.drop(['gbs'], axis = 1, inplace = True) #dont need this column anymore

KeyError: "['gbs'] not found in axis"

Extracting average prices for iphone models. We will need it in future to merge iPhones. Usually the iphones of different coulours cost the same, but not always.

In [158]:
abt['avg_model_price'] = abt.groupby(['iphone_model','iphone_storage'])['price_cleaned'].transform('mean')

In [159]:
abt['avg_model_price'] = round(abt['avg_model_price'],2)
abt[cellphones_filter]['avg_model_price'] #395 index had different prices for example

300     999.99
301    1099.99
302    1099.99
303     429.99
304     899.99
        ...   
395    1041.66
396     729.99
397     629.99
398     679.99
399     629.99
Name: avg_model_price, Length: 100, dtype: float64

In [160]:
abt[iphones_filtered].head()

,scraped_url,category,title,price,product_url,rating,review,model_id,iphone_storage,iphone_model,iphone_color,price_cleaned,reviews_number,rating_float,price_per_gb,avg_model_price
300,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 128GB Deep Purple iPhone 14 Pro Cellular...,Your Price: $999.99,https://www.abt.com/Apple-128GB-Deep-Purple-iP...,NaN,NaN,MQ0E3LL/A,128GB,14 Pro,Deep Purple,999.99,0,NaN,7.81,999.99
301,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 128GB Space Black iPhone 14 Pro Max Cell...,"Your Price: $1,099.99",https://www.abt.com/Apple-128GB-Space-Black-iP...,NaN,NaN,MQ8N3LL/A,128GB,14 Pro Max,Space Black,1099.99,0,NaN,8.59,1099.99
302,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 128GB Silver iPhone 14 Pro Max Cellular ...,"Your Price: $1,099.99",https://www.abt.com/Apple-128GB-Silver-iPhone-...,NaN,NaN,MQ8P3LL/A,128GB,14 Pro Max,Silver,1099.99,0,NaN,8.59,1099.99
303,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 64GB (PRODUCT)RED iPhone SE (3rd Generat...,Your Price: $429.99,https://www.abt.com/Apple-64GB-PRODUCT-RED-iPh...,NaN,NaN,NaN,64GB,SE (3rd Generation),(PRODUCT)RED,429.99,0,NaN,6.72,429.99
304,https://www.abt.com/Apple-Cell-Phones/bc/281-1...,Apple Cell Phones,Apple 256GB Starlight iPhone 14 Cellular Phone...,Your Price: $899.99,https://www.abt.com/Apple-256GB-Starlight-iPho...,NaN,NaN,MPW23LL/A,256GB,14,Starlight,899.99,0,NaN,3.52,899.99


Adding prices in CHF

In [161]:
usd_chf = 0.9046 #from bloomberg.com 25.05.23
abt['price_chf'] = round(abt.price_cleaned*usd_chf,2)

Droping of not needed columns: 

In [162]:
abt.drop(['scraped_url', 'price', 'rating', 'review'], axis = 1, inplace = True)

In [163]:
abt.head(2)

,category,title,product_url,model_id,iphone_storage,iphone_model,iphone_color,price_cleaned,reviews_number,rating_float,price_per_gb,avg_model_price,price_chf
0,Apple Desktop Computers,Apple Mac Mini Desktop Apple M2 Chip 8GB Unifi...,https://www.abt.com/Apple-Mac-Mini-Desktop-App...,MMFJ3LL/A,NaN,NaN,NaN,557.0,1,5.0,2.18,NaN,503.86
1,Apple Desktop Computers,"Apple Silver 24"" iMac M1 8-Core 8GB RAM 256GB ...",https://www.abt.com/Apple-Silver-24-iMac-M1-8-...,MGTF3LL/A,NaN,NaN,NaN,1299.0,4,5.0,5.07,NaN,1175.08


In [171]:
abt.to_csv('../data/abt_rudyka_stage3.csv') #saving the result

# Uploading to MariaDB

In [172]:
abt = pd.read_csv('../data/abt_rudyka_stage3.csv', sep =',', index_col = 0)
len(abt)

484

In [173]:
# Connect to MariaDB
try:
    conn = mariadb.connect(
        user="cip_user",
        password="cip_pw",
        host="127.0.0.1",             
        port=3306, 
        database="CIP"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

# Drop the table if it exists
cur.execute("DROP TABLE IF EXISTS abt_stage3")


# Define the table schema
table_schema = """
CREATE TABLE IF NOT EXISTS abt_stage3 (
    id INT AUTO_INCREMENT PRIMARY KEY,
    category VARCHAR(255),
    title VARCHAR(255),
    product_url VARCHAR(255),
    model_id VARCHAR(255),
    iphone_storage VARCHAR(255),
    iphone_model VARCHAR(255),
    iphone_color VARCHAR(255),
    price_cleaned VARCHAR(255),
    reviews_number VARCHAR(255),
    rating_float VARCHAR(255),
    price_per_gb VARCHAR(255),
    avg_model_price VARCHAR(255),
    price_chf VARCHAR(255)
)
"""

# Create the table
cur.execute(table_schema)

# Upload DataFrame data to the table
for key, row in abt.iterrows():
    insert_query = """
    INSERT INTO abt_stage3 
    (category,title, product_url, model_id, iphone_storage, iphone_model, iphone_color, price_cleaned, reviews_number, rating_float, price_per_gb, avg_model_price, price_chf) 
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
     # Convert NaN values to None (NULL)
    row = row.where(pd.notnull(row), None)
    
    cur.execute(insert_query, tuple(row))

# Commit the changes
conn.commit()

Checking

In [174]:
# Querying three titles
try:
    cur.execute("SELECT title FROM abt_stage3 LIMIT 5;")    
except mariadb.Error as e:
    print(f"Error: {e}")
    
for (title) in cur:
    print(f"Title: {title}")
    
#checking number of rows:
try:
    cur.execute("SELECT COUNT(title) FROM abt_stage3;") 
except mariadb.Error as e:
    print(f"Error: {e}")
    
result = cur.fetchone()  # Fetch the one result
count = result[0]  # Extract the count value from result

print(f"Total Count: {count}")

Title: ('Apple Mac Mini Desktop Apple M2 Chip 8GB Unified RAM 256GB SSD (Early 2023) - MMFJ3LL/A',)
Title: ('Apple Silver 24" iMac M1 8-Core 8GB RAM 256GB SSD, 7-Core GPU Retina 4.5K Desktop Computer (2021) - MGTF3LL/A',)
Title: ('Apple Blue 24" iMac M1 8-Core 8GB RAM 256GB SSD, 7-Core GPU Retina 4.5K Desktop Computer (2021) - MJV93LL/A',)
Title: ('Apple Mac Studio Desktop Apple M1 Ultra Chip 64GB Unified RAM 1TB SSD (Early 2022) - MJMW3LL/A',)
Title: ('Apple Green 24" iMac M1 8-Core 8GB RAM 256GB SSD, 7-Core GPU Retina 4.5K Desktop Computer (2021) - MJV83LL/A',)
Total Count: 484
